# Setup

## Main

In [ ]:
import getpass
import IPython.display
import json
import os
import requests
import pandas as pd 
from pyngrok import ngrok

## Data Management

### AWS S3

In [ ]:
from IPython.display import Image
import boto3  # boto3: high-level API
from botocore import UNSIGNED  # botocore: lower-level API and components
from botocore.config import Config
from question_answer.metadata.shared import DATA_DIRNAME, DOWNLOADED_DATA_DIRNAME

### Great Expectations + ZenML

#### Setup the Stack

In [ ]:
!zenml integration install great_expectations s3 dash -y

!zenml artifact-store register s3_store \
    --flavor=s3 \
    --path=s3://admirer-pica-zenml-greatexpectations

!zenml data-validator register great_expectations \
    --flavor=great_expectations

!zenml stack register admirer-pica_stack \
    -o default \
    -a s3_store \
    -dv great_expectations \
    --set

#### Libraries

In [ ]:
import great_expectations as ge

In [ ]:
from great_expectations.checkpoint.types.checkpoint_result import (  # type: ignore[import]
    CheckpointResult,
)

from zenml.integrations.constants import GREAT_EXPECTATIONS, SKLEARN
from zenml.integrations.great_expectations.steps import (
    GreatExpectationsProfilerParameters,
    GreatExpectationsProfilerStep,
    GreatExpectationsValidatorParameters,
    GreatExpectationsValidatorStep,
)
from zenml.integrations.great_expectations.visualizers import (
    GreatExpectationsVisualizer,
)
from zenml.pipelines import pipeline
from zenml.steps import BaseParameters, Output, step


from zenml.steps import (
    STEP_ENVIRONMENT_NAME,
    StepEnvironment,
)
from zenml.environment import Environment
from typing import cast


from zenml.config import DockerSettings


from zenml.post_execution import get_pipeline

## Deployment

In [ ]:
from app_gradio import app
from question_answer.answer import Pipeline

In [ ]:
!python3 training/wab.py --fetch

# Data Management

## AWS S3

In [ ]:
s3_bucket_name = "admirer-pica"  # objects are placed into buckets
s3_directory_path = "images"  # buckets can contain "folders" for organization
# we combine this information into a base URL format for the data:
s3_url = f"https://{s3_bucket_name}.s3.us-west-1.amazonaws.com/{s3_directory_path}"
s3_url

In [ ]:
person_idx = 1
image_idx = 1
img_url = f"{s3_url}/{str(person_idx).zfill(3)}_{str(image_idx).zfill(2)}.png"
print(img_url)
# Image(url=img_url, width=360)

In [ ]:
ADMIRER_RAW_DATA_DIRNAME = DATA_DIRNAME / "raw" / s3_bucket_name
ADMIRER_DL_DATA_DIRNAME = DOWNLOADED_DATA_DIRNAME / s3_bucket_name / s3_directory_path

In [ ]:
# spin up a client for communicating with s3 without authenticating ("UNSIGNED" activity)
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

!mkdir -p {ADMIRER_DL_DATA_DIRNAME}

s3.download_file(
    s3_bucket_name, s3_directory_path + "/001_01.png", f"{ADMIRER_DL_DATA_DIRNAME}/001_01.png")

# Image(filename=f"{ADMIRER_DL_DATA_DIRNAME}/001_01.png", width=720)

In [ ]:
s3_resource = boto3.resource('s3', config=Config(signature_version=UNSIGNED))


def download_s3_folder(bucket_name, s3_folder, local_dir=None):
    """Download the contents of a folder on S3, recursively.

    Parameters
    ----------
    bucket_name: the name of the s3 bucket
    s3_folder: the folder path in the s3 bucket
    local_dir: a relative or absolute directory path in the local file system
    """
    # from https://stackoverflow.com/questions/49772151/download-a-folder-from-s3-using-boto3
    bucket = s3_resource.Bucket(bucket_name)
    for obj in bucket.objects.filter(Prefix=s3_folder):
        target = obj.key if local_dir is None \
            else os.path.join(local_dir, os.path.relpath(obj.key, s3_folder))
        if not os.path.exists(os.path.dirname(target)):
            os.makedirs(os.path.dirname(target))
        if obj.key[-1] == '/':
            continue
        bucket.download_file(obj.key, target)
        
        
download_s3_folder(s3_bucket_name, s3_directory_path, ADMIRER_DL_DATA_DIRNAME)

In [ ]:
!find {ADMIRER_DL_DATA_DIRNAME} | head -n 20

## LabelStudio

### Configuring and connecting to the web server

In [ ]:
username = "admirer@localhost"
password = "moonshine"

%env LABEL_STUDIO_USERNAME={username}
%env LABEL_STUDIO_PASSWORD={password}

In [ ]:
config_file = ngrok.conf.DEFAULT_NGROK_CONFIG_PATH
config_file_exists =  os.path.exists(config_file)
config_file_contents = !cat {config_file}

auth_token_found = config_file_exists \
    and config_file_contents \
    and "authtoken" in config_file_contents[0] \
    and ": exit" not in config_file_contents  # state if interrupted

if not auth_token_found:
    print("Enter your ngrok auth token, which can be copied from https://dashboard.ngrok.com/auth")
    !ngrok authtoken {getpass.getpass()}

In [ ]:
LABEL_STUDIO_PORT = 8081
%env LABEL_STUDIO_PORT={LABEL_STUDIO_PORT}

https_tunnel = ngrok.connect(LABEL_STUDIO_PORT, bind_tls=True)
print(https_tunnel)

In [ ]:
# python3 -m venv label-env
# conda deactivate
# source label-env/bin/activate
# pip install -qqq label-studio
# export LABEL_STUDIO_PORT=8081
# label-studio start --port=$LABEL_STUDIO_PORT

In [ ]:
print(https_tunnel.public_url)
print("u:", username)
print("p:", password)

### Uploading Data

In [ ]:
"""
img_urls = []
for person_idx in range(1, 104):
    for image_idx in range(1, 13):
        img_urls.append(f"{s3_url}/{str(person_idx).zfill(3)}_{str(image_idx).zfill(2)}.png")

df = pd.DataFrame(img_urls, columns=["webcam"])
df.to_csv(str(ADMIRER_RAW_DATA_DIRNAME / "manifest.csv"), index=False)
"""

In [ ]:
print(ADMIRER_RAW_DATA_DIRNAME / "manifest.csv")
!cat {ADMIRER_RAW_DATA_DIRNAME}/manifest.csv | head -n 10

### Teardown

In [ ]:
# deactivate
# conda activate admirer

## Great Expectations + ZenML

### Define ZenML Steps


In [ ]:
class DataLoaderParameters(BaseParameters):
    reference_data: bool = True

@step
def importer(
        params: DataLoaderParameters,
) -> Output(dataset=pd.DataFrame, condition=bool):
    # Load labeled projects
    admirer_pica_path = "data/raw/admirer-pica/admirer-pica.json"
    df = pd.read_json(admirer_pica_path)
    dataset = df[:5]
    return dataset, params.reference_data

In [ ]:
# instantiate a builtin Great Expectations data profiling step
ge_profiler_params = GreatExpectationsProfilerParameters(
    expectation_suite_name="admirer-pica",
    data_asset_name="admirer-pica_test_df",
)
ge_profiler_step = GreatExpectationsProfilerStep(params=ge_profiler_params)


# instantiate a builtin Great Expectations data validation step
ge_validator_params = GreatExpectationsValidatorParameters(
    expectation_suite_name="admirer-pica",
    data_asset_name="admirer-pica_test_df",
)
ge_validator_step = GreatExpectationsValidatorStep(params=ge_validator_params)

In [ ]:
@step
def analyze_result(
    result: CheckpointResult,
) -> str:
    """Analyze the Great Expectations validation result and return a true/false value indicating
    whether it passed or failed."""
    step_env = cast(StepEnvironment, Environment()[STEP_ENVIRONMENT_NAME])
    pipeline_name = step_env.pipeline_name
    pipeline_run_id = step_env.pipeline_run_id
    step_name = step_env.step_name
    pipeline_context = f"Pipeline {pipeline_name}, with run {pipeline_run_id}, in step {step_name} produced the following output:\n\n"
    if result.success:
        message = pipeline_context + "Great Expectations data validation was successful!"
    else:
        message = pipeline_context + "Great Expectations data validation failed!"
    print(message)
    return message

### Define ZenML Pipelines

In [ ]:
docker_settings = DockerSettings(required_integrations=[GREAT_EXPECTATIONS])

@pipeline(enable_cache=False, settings={"docker": docker_settings})
def profiling_pipeline(
    importer, profiler
):
    """Data profiling pipeline for Great Expectations.

    The pipeline imports a reference dataset from a source then uses the builtin
    Great Expectations profiler step to generate an expectation suite (i.e.
    validation rules) inferred from the schema and statistical properties of the
    reference dataset.

    Args:
        importer: reference data importer step
        profiler: data profiler step
    """
    dataset, _ = importer()
    profiler(dataset)

In [ ]:
@pipeline(enable_cache=False, settings={"docker": docker_settings})
def validation_pipeline(
    importer, validator, checker
):
    """Data validation pipeline for Great Expectations.

    The pipeline imports a test data from a source, then uses the builtin
    Great Expectations data validation step to validate the dataset against
    the expectation suite generated in the profiling pipeline.

    Args:
        importer: test data importer step
        validator: dataset validation step
        checker: checks the validation results
    """
    dataset, condition = importer()
    results = validator(dataset, condition)
    message = checker(results)

### Run the pipelines

In [ ]:
profiling_pipeline(
    importer=importer(params=DataLoaderParameters(reference_data=True)),
    profiler=ge_profiler_step,
).run()

In [ ]:
validation_pipeline(
    importer=importer(params=DataLoaderParameters(reference_data=True)),
    validator=ge_validator_step,
    checker=analyze_result(),
).run()

### Post execution workflow

In [ ]:
def start_pipeline_visualizer(name: str):

    from zenml.integrations.dash.visualizers.pipeline_run_lineage_visualizer import (
        PipelineRunLineageVisualizer,
    )

    latest_run = get_pipeline(name).runs[-1]
    PipelineRunLineageVisualizer().visualize(latest_run)

In [ ]:
def visualize_results(pipeline_name: str, step_name: str) -> None:
    pipeline = get_pipeline(pipeline_name)
    last_run = pipeline.runs[-1]
    step = last_run.get_step(step=step_name)
    GreatExpectationsVisualizer().visualize(step)

In [ ]:
# start_pipeline_visualizer("profiling_pipeline")

In [ ]:
# start_pipeline_visualizer("validation_pipeline")

In [ ]:
# visualize_results("profiling_pipeline", "profiler")

In [ ]:
# visualize_results("validation_pipeline", "validator")

# Serve and Deploy
- Local/Local 
- Cloud Server/Cloud Server

In [ ]:
qa = Pipeline()

In [ ]:
example_img = "question_answer/tests/support/images/img.jpg"
example_question = "question_answer/tests/support/questions/question.txt"

print(qa.predict(example_img, example_question))

In [ ]:
frontend = app.make_frontend(qa.predict, flagging=True)

In [ ]:
frontend.launch(share=True, width="100%")

In [ ]:
%env API_URL={frontend.share_url + "/api"}

In [ ]:
response, = ! \
  (echo -n '{ "data": ["data:image/jpg;base64,'$(base64 -w0 -i question_answer/tests/support/images/img.jpg)'", "data:question/str;str,'$(cat question_answer/tests/support/questions/question.txt)'"] }') \
  | curl -s -X POST "${API_URL}/predict" -H 'Content-Type: application/json' -d @-
  
response

In [ ]:
print(json.loads(response)["data"][0])

In [ ]:
frontend.close()

# Serverless Backend (AWS Lambda)

## Build container image

In [ ]:
os.environ["LAMBDA_NAME"] = "admirer-backend"

In [ ]:
!docker build -t $LAMBDA_NAME . --file api_serverless/Dockerfile #--no-cache

In [ ]:
# export LAMBDA_NAME=admirer-backend
# docker run -p 9000:8080 $LAMBDA_NAME\:latest

In [ ]:
!curl -XPOST \
  "http://localhost:9000/2015-03-31/functions/function/invocations" \
  -d '{"image_url": "question_answer/tests/support/images/img.jpg", "question": "What color is my hair"}'

## Upload to the container registry

In [ ]:
# aws configure

In [ ]:
aws_account_id, = !aws sts get-caller-identity \
  --query "Account"
aws_region, = !aws configure get region 

os.environ["AWS_REGION"] = aws_region
os.environ["AWS_ACCOUNT_ID"] = aws_account_id.strip('"')

!echo $AWS_ACCOUNT_ID
!echo $AWS_REGION

In [ ]:
os.environ["ECR_URI"] = ".".join(
    [os.environ["AWS_ACCOUNT_ID"], "dkr", "ecr", os.environ["AWS_REGION"], "amazonaws.com"])

!echo $ECR_URI

In [ ]:
!aws ecr get-login-password --region $AWS_REGION \
  | docker login --username AWS --password-stdin $ECR_URI

In [ ]:
!aws ecr create-repository \
  --repository-name $LAMBDA_NAME \
  --image-scanning-configuration scanOnPush=true --image-tag-mutability MUTABLE \
  | jq -C

In [ ]:
os.environ["IMAGE_URI"] = "/".join([os.environ["ECR_URI"], os.environ["LAMBDA_NAME"]])

In [ ]:
!docker tag $LAMBDA_NAME\:latest $IMAGE_URI\:latest

In [ ]:
!docker push $IMAGE_URI\:latest

## Create a Lambda function

In [ ]:
os.environ["LAMBDA_ROLE_NAME"] = "lambda-role"

In [ ]:
!aws iam create-role \
  --role-name $LAMBDA_ROLE_NAME \
  --assume-role-policy-document '{"Version": "2012-10-17", "Statement": [{"Effect": "Allow", "Principal": {"Service": "lambda.amazonaws.com"}, "Action": "sts:AssumeRole"}]}' \
  | jq -C

In [ ]:
lambda_role_arn, = !aws iam get-role --role-name $LAMBDA_ROLE_NAME --output json | jq -r '.Role.Arn'
lambda_role_arn = lambda_role_arn.strip('"')

os.environ["LAMBDA_ROLE_ARN"] = lambda_role_arn
!echo $LAMBDA_ROLE_ARN

In [ ]:
# allow this IAM role to execute Lambdas
!aws iam attach-role-policy \
  --role-name $LAMBDA_ROLE_NAME \
  --policy-arn arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole

In [ ]:
# allow this IAM role to write to logs -- required and also important for debugging Lambdas
!aws iam attach-role-policy \
  --role-name $LAMBDA_ROLE_NAME \
  --policy-arn arn:aws:iam::aws:policy/AWSXRayDaemonWriteAccess

In [ ]:
!aws lambda create-function \
  --function-name $LAMBDA_NAME \
  --region $AWS_REGION \
  --package-type Image \
  --code ImageUri=$IMAGE_URI:latest \
  --role $LAMBDA_ROLE_ARN | jq -C

In [ ]:
!aws lambda update-function-configuration \
   --function-name $LAMBDA_NAME \
   --region $AWS_REGION \
   --timeout 60 \
   --memory-size 10240 | jq -C

In [ ]:
!aws lambda invoke \
  --function-name $LAMBDA_NAME \
  --invocation-type RequestResponse \
  --payload '{"image_url": "question_answer/tests/support/images/img.jpg", "question": "What color is my hair"}' \
  --cli-binary-format raw-in-base64-out lambda.out | jq -C

!cat lambda.out

## Add an HTTP endpoint with a URL

In [ ]:
!aws lambda create-function-url-config \
  --function-name $LAMBDA_NAME \
  --auth-type NONE \
  --cors '{"AllowOrigins": ["*"], "AllowCredentials": false}' \
  | jq -C

In [ ]:
# Careful here!!!
# """
!aws lambda add-permission \
 --function-name $LAMBDA_NAME \
 --action lambda:invokeFunctionUrl \
 --statement-id "open-access" \
 --principal "*" \
 --function-url-auth-type NONE | jq -C
# """

In [ ]:
lambda_url, = !aws lambda get-function-url-config --function-name $LAMBDA_NAME | jq .FunctionUrl
lambda_url = lambda_url.strip('"')

lambda_url

In [ ]:
image_url = "question_answer/tests/support/images/img.jpg"
question = "What color is my hair"

headers = {"Content-type": "application/json"}
payload = json.dumps({"image_url": image_url, "question": question})

response = requests.post(
  lambda_url, data=payload, headers=headers)
pred = response.json()["pred"]

print(pred)

## Connect AWS with Gradio

In [ ]:
serverless_backend = app.PredictorBackend(url=lambda_url)

In [ ]:
frontend_serverless_backend = app.make_frontend(serverless_backend.run, flagging=True)
frontend_serverless_backend.launch(share=True)

In [ ]:
frontend_serverless_backend.close()

# Serving through Ngrok

In [ ]:
frontend = frontend #frontend_serverless_backend
frontend.local_url

In [ ]:
!curl -X POST {frontend.local_url}api/predict

In [ ]:
config_file = ngrok.conf.DEFAULT_NGROK_CONFIG_PATH
config_file_exists =  os.path.exists(config_file)
config_file_contents = !cat {config_file}

auth_token_found = config_file_exists \
    and config_file_contents \
    and "authtoken" in config_file_contents[0] \
    and ": exit" not in config_file_contents  # state if interrupted

if not auth_token_found:
    print("Enter your ngrok auth token, which can be copied from https://dashboard.ngrok.com/auth")
    !ngrok authtoken {getpass.getpass()}

In [ ]:
ADMIRER_PORT = frontend.server_port
ADMIRER_PORT

In [ ]:
# https_tunnel = ngrok.connect(ADMIRER_PORT, bind_tls=True)
# print(https_tunnel)

# Load Testing with Locust

## Running the load test

In [ ]:
!locust --locustfile=locust_http_user.py \
  --headless \
  --users=10 \
  --spawn-rate=1 \
  --run-time=2m \
  --host=https://joiajq6syp65ueonto4mswttzu0apfbi.lambda-url.us-west-1.on.aws \
  --html=locust_report.html \
  --csv=locust_report

## Viewing the results

In [ ]:
!ls -lh locust_report*

In [ ]:
IPython.display.HTML("locust_report.html")

## Analyzing load test data programmatically

In [ ]:
csv_path = "locust_report_stats_history.csv"
results = pd.read_csv(csv_path)
results["Timestamp"] = pd.to_datetime(results["Timestamp"], unit="s")
results.tail()

In [ ]:
request_columns = ["Total Request Count", "Total Failure Count", "User Count"]
results.plot(x="Timestamp", y=request_columns, subplots=True, sharey=True);

In [ ]:
response_columns = ["Total Average Response Time", "Total Max Response Time"]
results.plot(x="Timestamp", y=response_columns);

In [ ]:
results.groupby("Total Median Response Time").describe()